In [2]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.19


In [3]:
# https://docs.sqlalchemy.org/en/20/tutorial/engine.html

from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [4]:
# Getting a connection
# https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.htmlhttps://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html

from sqlalchemy import text

with engine.connect() as conn:
    # textual SQL statement
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

    # The transaction is not committed automatically

2023-12-13 16:26:02,005 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:26:02,007 INFO sqlalchemy.engine.Engine select 'hello world'
2023-12-13 16:26:02,009 INFO sqlalchemy.engine.Engine [generated in 0.00380s] ()
[('hello world',)]
2023-12-13 16:26:02,010 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
# commit changes in a `as you go` fashion
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-12-13 16:26:02,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:26:02,025 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-12-13 16:26:02,026 INFO sqlalchemy.engine.Engine [generated in 0.00348s] ()
2023-12-13 16:26:02,028 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 16:26:02,029 INFO sqlalchemy.engine.Engine [generated in 0.00123s] [(1, 1), (2, 4)]
2023-12-13 16:26:02,032 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
# `begin once`` fashion
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-12-13 16:26:02,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:26:02,049 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 16:26:02,049 INFO sqlalchemy.engine.Engine [cached since 0.0215s ago] [(6, 8), (9, 10)]
2023-12-13 16:26:02,051 INFO sqlalchemy.engine.Engine COMMIT


```
2023-12-13 15:27:13,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
```
BEGIN (implicit) means it did not actually send any command.

In [7]:
# select
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-12-13 16:26:02,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:26:02,066 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 16:26:02,067 INFO sqlalchemy.engine.Engine [generated in 0.00334s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-12-13 16:26:02,069 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
# Sending Parameters
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-12-13 16:26:03,262 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:26:03,264 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-12-13 16:26:03,265 INFO sqlalchemy.engine.Engine [generated in 0.00357s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-12-13 16:26:03,269 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
# Sending Multiple Parameters
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()
    # executemany doesn’t support returning

2023-12-13 16:27:08,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:27:08,631 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 16:27:08,632 INFO sqlalchemy.engine.Engine [cached since 66.61s ago] [(11, 12), (13, 14)]
2023-12-13 16:27:08,634 INFO sqlalchemy.engine.Engine COMMIT


## ORM Session

In [10]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

    # For non-ORM calls, ORM Session does nothing different than the Connection

2023-12-13 16:29:46,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 16:29:46,120 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-12-13 16:29:46,121 INFO sqlalchemy.engine.Engine [generated in 0.00083s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2023-12-13 16:29:46,123 INFO sqlalchemy.engine.Engine ROLLBACK
